In [7]:
training_article = 'money_190101200531.csv'
test_article = 'all_200601200625.csv' # 사용할 본문 데이터 파일 이름
#labeling_point = 0.006 # 라벨링 기준값

In [2]:
import re
def modify(text):
    if '[ⓒ' in text:
        index = text.find('[ⓒ')
        text = text[:index]
    while 'googletag.display' in text:
        index = text.find('googletag.display')        
        index2 = text.find(';',index)
        text = text[:index]+' '+text[index2+1:]
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' #이메일 주소 제거
    text = re.sub(pattern=pattern, repl=' ',string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' #한글 자모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]' #특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    return text

In [3]:
#주가 데이터의 라벨링 과정
import csv
f = open('stock.csv','r', encoding='utf-8-sig')
rdf = csv.reader(f)
label ={}
for line in rdf:
    date = line[0][:4]+line[0][5:7]+line[0][8:10]
    number = float(line[6])
    if number > 0.0077:
        number = 1
    elif number < -0.005:
        number = -1
    else:
        number = 0
    label[date] = number
f.close()

In [ ]:
# 1. 해당 날짜의 전체 본문을 합쳐서 라벨링하는 코드
"""
text = {}
with open(training_article,'r',encoding='utf-8-sig') as f2:
        rdf2 = csv.reader(f2)
        for line in rdf2:
            date = line[1][:4]+line[1][5:7]+line[1][8:10]
            if date in label:
                if date in text:
                    text[date] = text[date]+' ' +modify(line[2])
                else:
                    text[date] = modify(line[2])  
training_data = []
for date in text:
    training_data.append([text[date],label[date],date])

text2 = {}    
with open(test_article,'r',encoding='utf-8-sig') as f3:
        rdf3 = csv.reader(f3)
        for line in rdf3:
            date = line[1][:4]+line[1][5:7]+line[1][8:10]
            if date in label:
                if date in text2:
                    text2[date] = text2[date]+' ' +modify(line[2])
                else:
                    text2[date] = modify(line[2])  
test_data = []
for date in text2:
    test_data.append([text2[date],label[date],date])
"""

In [8]:
# 2. 기사 본문을 각각 라벨링하는 코드
training_data = []
test_data = []    
with open(training_article,'r',encoding='utf-8-sig') as f2:
        rdf2 = csv.reader(f2)
        for line in rdf2:
            date = line[1][:4]+line[1][5:7]+line[1][8:10]
            if date in label:
                training_data.append([modify(line[2]),label[date],date])
with open(training_article2,'r',encoding='utf-8-sig') as f2:
        rdf2 = csv.reader(f2)
        for line in rdf2:
            date = line[1][:4]+line[1][5:7]+line[1][8:10]
            if date in label:
                training_data.append([modify(line[2]),label[date],date])

                             
with open(test_article,'r',encoding='utf-8-sig') as f3:
        rdf3 = csv.reader(f3)
        for line in rdf3:
            date = line[1][:4]+line[1][5:7]+line[1][8:10]
            if date in label:
                test_data.append([modify(line[2]),label[date],date])
    
# training_data : 날짜 정보가 없는 training_data
# training_data2 : 날짜 정보가 있는 training_data
# test_data : 날짜 정보가 없는 test_data
# test_data2 : 날짜 정보가 있는 test_data

In [9]:
# data를 셔플링 해주는 과정
from random import shuffle
for i in range(5):
    shuffle(training_data)
    shuffle(test_data)

In [11]:
# data를 토크나이징 하는 과정(단어만 추출하여 리스트 형태로 변환)
from konlpy.tag import Okt
import nltk

okt = Okt()

def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc,norm=True,stem=True)]
print('done')
training = [(tokenize(data[0]),data[1],data[2]) for data in training_data]#[(['단어/형태',...,]0 or 1), ...]
print('training is done')
test = [(tokenize(data[0]),data[1],data[2]) for data in test_data]
print('test is done')

done
test is done


In [12]:
# 생성된 training, test 데이터를 pickle의 형태로 저장
import pickle
import os
path = os.getcwd()+'/pickles/'
with open(path+'19training.pickle', 'wb') as f:
    pickle.dump(training, f)
with open(path+'19test.pickle', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
# training.pickle : summarize 하지 않은 기사 본문을 각각 라벨링한 데이터 (당일 기사로 다음날 주가)
# 2training.pickle : summarize 하지 않은 기사 본문을 날짜마다 합쳐서 라벨링한 데이터
# 3training.pickle : 40%로 summarize 한 기사 본문을 각각 라벨링한 데이터
# 4training.pickle : 40%로 summarize 한 기사 본문을 날짜마다 합쳐서 라벨링한 데이터
# 5training.pickle : summarize 하지 않은 기사 본문을 당일의 종가등락율로 각각 라벨링한 데이터
# 6training.pickle : summarize 하지 않은 기사 본문을 전일의 종가등락율로 각각 라벨링한 데이터
# 7training.pickle : summarize 하지 않은 기사 본문을 다음날의 시가등락율로 각각 라벨링한 데이터
# 8training.pickle : summarize 하지 않은 기사 본문을 당일의 전일 대비 종가등락율로 각각 라벨링한 데이터(final_articles 사용)
# 9training.pickle : summarize 하지 않은 기사 본문을 당일의 시가대비 종가등락율로 각각 라벨링한 데이터(final_articles 사용)